<a href="https://colab.research.google.com/github/cmlins/recuperacaoDeInformacao/blob/master/proc_consulta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import math

from bs4 import BeautifulSoup as bs
from bs4 import UnicodeDammit

In [520]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [521]:
df_ed = pd.read_csv('/content/drive/My Drive/dfs_tot (1).csv')
df_ed.head()

,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN,Documento
0,1808: Como uma rainha,Laurentino Gomes,384,Globo Livros,(Português),Capa Comum,978-8525057518,/content/drive/My Drive/9periodo/Recuperação...
1,Box - As Melhores Aventuras Do Sítio Do Picapa...,Monteiro Lobato,640,Nova Fronteira,(Português),Capa dura,978-8520943519,/content/drive/My Drive/9periodo/Recuperação...
2,Cotoco. O Diário De Um Garoto De 13 Anos,John Van de Ruit,390,Intrínseca,(Português),Capa Comum,978-8598078854,/content/drive/My Drive/9periodo/Recuperação...
3,"Doctor Who: 12 Doutores, 12 histórias",Alex Scarrow,480,Fantástica Rocco,(Português),Capa Comum,978-8568263044,/content/drive/My Drive/9periodo/Recuperação...
4,Freud (1900) A interpretação dos sonhos: Obras...,Sigmund Freud,736,Companhia das Letras,(Português),Capa dura,978-8535932218,/content/drive/My Drive/9periodo/Recuperação...


In [522]:
table_1.at[1, 'Documento']

'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/Box - As Melhores Aventuras Do Sítio Do Picapau Amarelo Monteiro Lobato Amazon.com.br.html'

# Funções

## Parse HTML

In [0]:
symbolsToReplace = [
    '(', ')', '!', '?', ':', ';', '*', '.', ',', '★', '|', '+', '[', ']', '{', '}', '/', 'ª', 'º', '°', '-',
    '%', '—', '@', '#',  '\"','\'', '<', '>', '=', '´', '`', '“', '$', '&', '’', '¡', '€', 'µ', '¦',
    '\\', '®', '™', '”', '…', '‘', '•', '😍','😙', '❤', '–', '⭐️', '_', '️⭐', '😉', '👏', '¹', '²', '³',
    "£", '¢', '¬', '§', '~', '^', '×', '÷',
    '\u200c', '\u200f', '\u200e', '\t', '\xa0', '\x03', '\u0301', '\u0303'
]

allowedSmallTokens = [
  'fio', 'box', 'god', 'key', 'bit', 'led', 'faq', 'dog', 'sim', 'ovo', 'pão', 'aba', 'new'
]

In [0]:
def getAndCleanHTML(filepath):
  
  page = open(filepath, "rb")

  page = page.read()

  clearPage = UnicodeDammit.detwingle(page)
  
  doc = bs(clearPage, 'lxml')
  
  for script in doc(["script", "style"]):
    script.extract()
  
  docText = doc.get_text(' ')
  
  docText = docText.translate({ord(ch): None for ch in '0123456789'})
  
  docText = docText.lower()
  
  docText = docText.replace("e-book", "ebook")

  docText = docText.replace("blu-ray", "bluray")
  
  for symbol in symbolsToReplace:
    docText = docText.replace(symbol, ' ')
  
  docText = docText.replace("  ", ' ')
  
  lines = (line.strip() for line in docText.splitlines())
  
  chunks = (phrase.strip() for line in lines for phrase in line.split(' '))

  text = ' '.join(chunk for chunk in chunks if chunk)
  
  return text

## Searching Index

In [0]:
def handlingQuery(query):
  hq = query.split()
  return hq

In [0]:
def getValues(query):
  rl = list()
  for i in query:
    wl = query[i].split(' ')
    for w in wl:
      rl.append(w)
  return rl

In [0]:
def free_search(q, indice):
  hq = handlingQuery(q)
  df_search = pd.DataFrame()
  for q in hq:
    for w in indice['Terms'].values:
      if (w in q) or (w == q):
        df_search = df_search.append(indice[indice['Terms'] == w], ignore_index=True)
  return df_search

In [0]:
def structured_search(sq, indice):
  df_search = pd.DataFrame()
  for i in sq:
    values_sq = sq[i].split(' ')
    for v in values_sq:
      for r in range(len(df_campos)):
        sr = df_campos.at[r, 'Field Text'].split('.')
        if (sr[0] in v) and (sr[1] == i):
          df_search = df_search.append(df_campos[df_campos['Field Text'] == sr[0] + '.' + sr[1]], ignore_index=True)
  return df_search

In [0]:
def docs_list_OR(df):
  postings = list()
  for r in df.index:
    postings.append(df.loc[r,'Posting'])

  for i in range(len(postings)):
    new_postings = postings[i].split(', ')
    new_postings[0] = new_postings[0][1:]
    new_postings[len(new_postings)-1] = new_postings[len(new_postings)-1][:-1]
    postings[i] = new_postings

  p = list()
  for i in range(len(postings)):
    for j in range(len(postings[i])):
      p.append(postings[i][j])

  p = list(dict.fromkeys(p))
  return p

In [0]:
def docs_list_AND(df):
  aux = list()
  postings_list = list()
  #colocar postings em listas
  for r in df.index:
    aux = df.at[r, 'Posting'].split(', ')
    aux[0] = aux[0][1:]
    aux[len(aux)-1] = aux[len(aux)-1][:-1]
    postings_list.append(aux)

  #iterar sobre listas de postings buscando itens iguais
  postings = list()
  for i in range(len(postings_list)):
    for j in range(i+1, len(postings_list)):
      for k in range(len(postings_list[i])):
        for l in range(len(postings_list[j])):
          if postings_list[i][k] == postings_list[j][l]:
            postings.append(postings_list[i][k])
            postings_list[i][j] = ' '

  return postings

## Calc Scores

In [0]:
def calc_tfidf(df_freq):
  for r in df_freq.index:
    df_freq.at[r, 'Freq'] = math.log(df_arquivos.shape[0]/df_freq.at[r, 'Freq'],10)
    for c in range(2, len(df_freq.columns)):
      df_freq.iloc[r, c] = np.multiply(df_freq.at[r, 'Freq'], df_freq.iloc[r, c])

  return df_freq

In [0]:
def calc_freq(dl, df_search, tipo_busca, q):
  df_freq = pd.DataFrame(columns=dl)
  if tipo_busca == 'livre':
    df_freq = pd.concat([df_search['Terms'], df_search['Freq'], df_freq], axis=1)
    queryl = handlingQuery(q)
  else:
      queryl = getValues(q)
      df_freq = pd.concat([df_search['Field Text'], df_search['Freq'], df_freq], axis=1)

  for r in df_freq.index:
    for c in range(2, len(df_freq.columns)):
      df_freq.iloc[r, c] = 0

  df_freq.iloc[:, 1:] = df_freq.iloc[:, 1:].astype(float)

  for i in range(len(dl)):
    doc = getAndCleanHTML(df_arquivos.loc[int(dl[i]),'0'])
    doc = doc.lower()
    doc = doc.split()
    for w in doc:
      for qw in range(len(queryl)):
        if w == queryl[qw]:
          df_freq.at[qw, dl[i]] += 1.0

  for r in df_freq.index:
    for c in range(2, len(df_freq.columns)):
      if df_freq.at[r,df_freq.columns[c]] !=0:
        df_freq.at[r,df_freq.columns[c]] = math.log10(df_freq.at[r, df_freq.columns[c]])
  
  return df_freq

## Calc Ranking

In [0]:
def calc_rank(df_freq, hq, tipo_busca):
  if tipo_busca == 'livre':
    df_freq.at[len(df_freq.index),'Terms'] = 'cosine'
    df_freq.index = df_freq['Terms']
  else:
    df_freq.at[len(df_freq.index),'Field Text'] = 'cosine'
    df_freq.index = df_freq['Field Text']

  df_freq = df_freq.iloc[:, 1:]

  dv = []
  qv = []
  for q in hq:
    qv.append(1)

  for c in range(1, len(df_freq.columns)):
    for r in range(len(df_freq.index[:-1])):
      dv.append(df_freq.iloc[r,c])
    df_freq.iloc[len(df_freq.index)-1, c] = (np.dot(dv, qv) / (np.linalg.norm(dv) * np.linalg.norm(qv)))
    dv = []

  df_rank = df_freq.sort_values(by='cosine', axis=1, ascending=False)
  df_rank = df_rank.drop(['Freq'], axis=1)
  ranking = df_rank.columns.tolist()
  ranking = ranking[:10]
  return ranking

In [0]:
def ranking_dictionary(r):
  rd_list = list()
  for i in r:
    path = df_arquivos.at[int(i), '0']
    row, = df_ed[df_ed['Documento'] == path].index
    titulo = df_ed.at[row, 'Título']
    autor = df_ed.at[row, 'Autor']
    rd = {
        'title': titulo,
        'author': autor
    }
    rd_list.append(rd)

  return rd_list

# Main

## Índice

In [535]:
df_indice = pd.read_csv('/content/drive/My Drive/indice_invertido_all_view_pages.csv')
df_indice.head(n=10)

,Terms,Freq,Posting,Compress Posting,Gamma Code
0,abaixo,38,"[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
1,abordagem,3,"[7, 64, 74]","[7, 57, 10]","[11011, 11111011001, 1110010]"
2,abrir,20,"[82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 9...","[82, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[1111110010010, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,abuso,8,"[0, 1, 3, 4, 5, 6, 7, 8]","[0, 1, 2, 1, 1, 1, 1, 1]","[0, 0, 100, 0, 0, 0, 0, 0]"
4,acabam,6,"[6, 49, 52, 55, 56, 57]","[6, 43, 3, 3, 1, 1]","[11010, 11111001011, 101, 101, 0, 0]"
5,acabamento,19,"[5, 9, 10, 11, 12, 13, 14, 15, 16, 49, 50, 51,...","[5, 4, 1, 1, 1, 1, 1, 1, 1, 33, 1, 1, 1, 1, 1,...","[11001, 11000, 0, 0, 0, 0, 0, 0, 0, 1111100000..."
6,acabou,8,"[89, 92, 95, 96, 98, 99, 100, 101]","[89, 3, 3, 1, 2, 1, 1, 1]","[1111110011001, 101, 101, 0, 100, 0, 0, 0]"
7,acaso,13,"[1, 13, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36...","[1, 12, 14, 1, 1, 1, 1, 1, 1, 1, 1, 1, 50]","[0, 1110100, 1110110, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,aceita,10,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 84]","[0, 1, 1, 1, 1, 1, 1, 1, 1, 76]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1111110001100]"
9,aceitamos,10,"[27, 28, 29, 30, 31, 32, 33, 34, 35, 36]","[27, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[111101011, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [536]:
df_arquivos = pd.read_csv('/content/drive/My Drive/urls_df.csv')
df_arquivos.head()

,0
0,/content/drive/My Drive/9periodo/Recuperação...
1,/content/drive/My Drive/9periodo/Recuperação...
2,/content/drive/My Drive/9periodo/Recuperação...
3,/content/drive/My Drive/9periodo/Recuperação...
4,/content/drive/My Drive/9periodo/Recuperação...


In [537]:
df_campos = pd.read_csv('/content/drive/My Drive/field_texts.csv')
df_campos.head()

,word,Field Text,Freq,Posting,Compress Posting
0,0,1808.title,1,[0],[0]
1,1,1900.title,1,[4],[4]
2,0,1967.publishing,1,[29],[29]
3,1,1984.publishing,1,[32],[32]
4,2,1987.publishing,1,[34],[34]


## Busca livre

In [538]:
fq = 'diário garoto'
df_fs = free_search(fq,df_indice)
df_fs.head(n=10)

,Terms,Freq,Posting,Compress Posting,Gamma Code
0,diário,30,"[1, 2, 9, 10, 11, 12, 13, 14, 15, 16, 19, 27, ...","[1, 1, 7, 1, 1, 1, 1, 1, 1, 1, 3, 8, 1, 1, 1, ...","[0, 0, 11011, 0, 0, 0, 0, 0, 0, 0, 101, 111000..."
1,garoto,13,"[2, 18, 53, 56, 82, 86, 87, 89, 90, 91, 92, 96...","[2, 16, 35, 3, 26, 4, 1, 2, 1, 1, 1, 4, 3]","[100, 111100000, 11111000011, 101, 111101010, ..."


In [539]:
dl_fs = docs_list_OR(df_fs)
print(dl_fs)

['1', '2', '9', '10', '11', '12', '13', '14', '15', '16', '19', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '38', '52', '55', '64', '65', '66', '69', '90', '92', '18', '53', '56', '82', '86', '87', '89', '91', '96', '99']


### Usando a frequência (tf)

In [540]:
df_f_fs = calc_freq(dl_fs, df_fs, 'livre', fq)
df_f_fs.head(n=10)

,Terms,Freq,1,2,9,10,11,12,13,14,15,16,19,27,28,29,30,31,32,33,34,35,36,38,52,55,64,65,66,69,90,92,18,53,56,82,86,87,89,91,96,99
0,diário,30.0,0.0,0.60206,0.477121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.30103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.778151,0.477121,0.0,0.30103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,garoto,13.0,0.0,0.60206,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [541]:
r_f_fs = calc_rank(df_f_fs, handlingQuery(fq), 'livre')
print(r_f_fs)

['2', '9', '27', '66', '69', '92', '1', '10', '11', '12']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


In [542]:
rd_f_fs = ranking_dictionary(r_f_fs)
print(rd_f_fs)

[{'title': 'Cotoco. O Diário De Um Garoto De 13 Anos', 'author': 'John Van de Ruit'}, {'title': 'Alice No País Das Maravilhas (Classic Edition)', 'author': 'Carroll,Lewis'}, {'title': 'A Casa Dos Budas Ditosos', 'author': 'João Ubaldo Ribeiro'}, {'title': 'O DIÁRIO DE ANNE FRANK', 'author': 'FRANK, ANNE'}, {'title': 'PRÓLOGO, ATO, EPÍLOGO', 'author': 'MONTENEGRO, FERNANDA'}, {'title': 'Luccas Neto em os aventureiros', 'author': 'Luccas Neto'}, {'title': 'Box - As Melhores Aventuras Do Sítio Do Picapau Amarelo', 'author': 'Monteiro Lobato'}, {'title': 'Steve Jobs - A Biografia', 'author': 'Isaacson,Walter, Isaacson,Walter'}, {'title': 'Vade Mecum Compacto   - Brochura  - 21ª Ed. 2019', 'author': 'Editora Saraiva'}, {'title': 'It - A Coisa', 'author': 'King,Stephen'}]


### Usando tf-idf

In [543]:
df_tfidf_fs = calc_freq(dl_fs, df_fs, 'livre', fq)
df_tfidf_fs = calc_tfidf(df_tfidf_fs)
df_tfidf_fs.head(n=10)

,Terms,Freq,1,2,9,10,11,12,13,14,15,16,19,27,28,29,30,31,32,33,34,35,36,38,52,55,64,65,66,69,90,92,18,53,56,82,86,87,89,91,96,99
0,diário,0.531479,0.0,0.319982,0.25358,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.159991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.413571,0.25358,0.0,0.159991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,garoto,0.894657,0.0,0.538637,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [544]:
r_tfidf_fs = calc_rank(df_tfidf_fs, handlingQuery(fq), 'livre')
print(r_tfidf_fs)

['2', '66', '9', '27', '69', '92', '1', '10', '11', '12']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


In [545]:
rd_tfidf_fs = ranking_dictionary(r_tfidf_fs)
print(rd_tfidf_fs)

[{'title': 'Cotoco. O Diário De Um Garoto De 13 Anos', 'author': 'John Van de Ruit'}, {'title': 'O DIÁRIO DE ANNE FRANK', 'author': 'FRANK, ANNE'}, {'title': 'Alice No País Das Maravilhas (Classic Edition)', 'author': 'Carroll,Lewis'}, {'title': 'A Casa Dos Budas Ditosos', 'author': 'João Ubaldo Ribeiro'}, {'title': 'PRÓLOGO, ATO, EPÍLOGO', 'author': 'MONTENEGRO, FERNANDA'}, {'title': 'Luccas Neto em os aventureiros', 'author': 'Luccas Neto'}, {'title': 'Box - As Melhores Aventuras Do Sítio Do Picapau Amarelo', 'author': 'Monteiro Lobato'}, {'title': 'Steve Jobs - A Biografia', 'author': 'Isaacson,Walter, Isaacson,Walter'}, {'title': 'Vade Mecum Compacto   - Brochura  - 21ª Ed. 2019', 'author': 'Editora Saraiva'}, {'title': 'It - A Coisa', 'author': 'King,Stephen'}]


## Busca estruturada

In [546]:
sq = {
    'title': 'sítio do picapau amarelo',
    'author': 'monteiro lobato',
    'isbn': '978-8520943519'
}

df_ss = structured_search(sq, df_campos)
df_ss.head(n=10)

,word,Field Text,Freq,Posting,Compress Posting
0,306,sítio.title,1,[1],[1]
1,58,capa.title,1,[45],[45]
2,253,picapau.title,1,[1],[1]
3,13,amarelo.title,1,[1],[1]
4,147,monteiro.author,1,[1],[1]
5,121,lobato.author,1,[1],[1]
6,1,8520943519.isbn,1,[1],[1]


In [547]:
dl_ss = docs_list_OR(df_ss)
print(dl_ss)

['1', '45']


### Usando a frequência (tf)

In [548]:
df_f_ss = calc_freq(dl_ss, df_ss, 'estruturada', sq)
df_f_ss.head(n=10)

,Field Text,Freq,1,45
0,sítio.title,1.0,0.954243,0.000000
1,capa.title,1.0,1.491362,1.113943
2,picapau.title,1.0,0.778151,0.000000
3,amarelo.title,1.0,0.778151,0.000000
4,monteiro.author,1.0,1.278754,0.000000
5,lobato.author,1.0,1.301030,0.000000
6,8520943519.isbn,1.0,0.000000,0.000000


In [549]:
r_f_ss = calc_rank(df_f_ss, getValues(sq), 'estruturada')
print(r_f_ss)

['1', '45']


In [550]:
rd_f_ss = ranking_dictionary(r_f_ss)
print(rd_f_ss)

[{'title': 'Box - As Melhores Aventuras Do Sítio Do Picapau Amarelo', 'author': 'Monteiro Lobato'}, {'title': 'Manual do Arquiteto Descalço - Capa Dura', 'author': 'Johan Van Lengen'}]


### Usando tf-idf

In [551]:
df_tfidf_ss = calc_freq(dl_ss, df_ss, 'estruturada', sq)
df_tfidf_ss = calc_tfidf(df_tfidf_ss)
df_tfidf_ss.head(n=10)

,Field Text,Freq,1,45
0,sítio.title,2.0086,1.916692,0.000000
1,capa.title,2.0086,2.995549,2.237467
2,picapau.title,2.0086,1.562995,0.000000
3,amarelo.title,2.0086,1.562995,0.000000
4,monteiro.author,2.0086,2.568505,0.000000


In [552]:
r_tfidf_ss = calc_rank(df_tfidf_ss, getValues(sq), 'estruturada')
print(r_tfidf_ss)

['1', '45']


In [553]:
rd_tfidf_ss = ranking_dictionary(r_tfidf_ss)
print(rd_tfidf_ss)

[{'title': 'Box - As Melhores Aventuras Do Sítio Do Picapau Amarelo', 'author': 'Monteiro Lobato'}, {'title': 'Manual do Arquiteto Descalço - Capa Dura', 'author': 'Johan Van Lengen'}]
